In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import matplotlib.pyplot as plt
from keras.models import Model
from matplotlib import pyplot
from keras.layers import Input
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
import matplotlib.pyplot as plt
import keras
from keras import optimizers
#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [ ]:
DATADIR = '/content/drive/My Drive/UCI_HAR_Dataset'

In [ ]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [ ]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

In [ ]:
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    filename = f'/content/drive/My Drive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [ ]:
def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, y_train, X_test,  y_test

In [ ]:
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [ ]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
X_train, Y_train, X_test,  Y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
#n_classes  = 6
print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [ ]:

model = Sequential()
model.add(Conv1D(36, 15, padding='same',activation='relu',strides=1, input_shape=(128,9)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(24, 13, padding='same',activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(12, 5, padding='same',activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 128, 36)           4896      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 64, 36)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 24)            11256     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 24)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 12)            1452      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 16, 12)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 12)            4

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist1=model.fit(X_train,
          Y_train,
          batch_size=64,
          epochs=30, 
            callbacks=[es],
               validation_split=0.2)

Epoch 1/30
92/92 [==============================] - 3s 35ms/step - loss: 0.7013 - accuracy: 0.7089 - val_loss: 1.2648 - val_accuracy: 0.4813
Epoch 2/30
92/92 [==============================] - 3s 33ms/step - loss: 0.1665 - accuracy: 0.9373 - val_loss: 0.6955 - val_accuracy: 0.7145
Epoch 3/30
92/92 [==============================] - 3s 32ms/step - loss: 0.1343 - accuracy: 0.9475 - val_loss: 0.2461 - val_accuracy: 0.9177
Epoch 4/30
92/92 [==============================] - 3s 32ms/step - loss: 0.1143 - accuracy: 0.9548 - val_loss: 0.4096 - val_accuracy: 0.8545
Epoch 00004: early stopping


In [ ]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 512        0  ...                   0                 1
SITTING                  0      412  ...                   0                 1
STANDING                 0       70  ...                   0                 0
WALKING                  1        0  ...                  30                 1
WALKING_DOWNSTAIRS       0        0  ...                 418                 2
WALKING_UPSTAIRS         5        0  ...                  22               442

[6 rows x 6 columns]
